In [1]:
%%html
<style>
table {float:left}
</style>

# El Molino Traffic Collision Stats
This analysis uses the Pasadena Traffic Collision stats, which Pasadena provides at https://data.cityofpasadena.net/datasets/85f49ea583c24056968bee6e28162da4/explore. Pasadena last updated the data 5/5/22 and the data set was downloaded 7/12/23.

This analysis takes two approaches:
1. Longitude: El Molino is a North South street that stays within -118.1373 and -118.1367 longitude. The longitude approach aggregates the data that falls within these two longitudes.
2. Street and Cross Street: The Pasadena Traffic Collision data identifies a street and cross street where the accident occured. The Street/Cross Street approach aggregates the data that has El Molino Ave listed in either the Street or Cross Street columns.

## Results:
Between 1/11/08 to 12/24/20:

Approach | Total Collisions | Number Injured | Number Killed
:-: | :-: | :-: | :-:
Longitude | 489 | 289 | 2
Street/Cross Street | 574 | 307 | 2

## Import libraries and Data

In [1]:
import pandas as pd
import plotly.express as px
df = pd.read_pickle('Traffic_Collisions.pkl')

## Longitude Analysis

In [2]:
el_molino_lon_df = df.loc[(df['lon'] >= -118.1373) & (df['lon'] <= -118.1367)]

# aggregate
print(f'Total collisions {len(el_molino_lon_df)}')
display(el_molino_lon_df.agg({'NoInjured' : ['sum'], 'NoKilled': ['sum'], 'datetime': ['min', 'max']}))

# plot
fig = px.density_mapbox(el_molino_lon_df, lat='lat', lon='lon', radius=4,
                        center=dict(lat=34.15127, lon=-118.1369), zoom=13,
                        mapbox_style="stamen-terrain", height=1000, )
#fig.show()

# Cause
el_molino_lon_df.Cause.value_counts()

Total collisions 489


,NoInjured,NoKilled,datetime
sum,289.0,2.0,NaT
min,NaN,NaN,2008-01-11
max,NaN,NaN,2020-12-24


Traffic Signals and Signs                                      149
Unsafe Speed                                                    65
Unknown                                                         65
Improper Turning                                                55
Auto R/W Violation                                              42
Driving Under Influence                                         17
Pedestrian Violation                                            15
Unsafe Starting or Backing                                      15
Not Stated                                                      13
Wrong Side of Road                                              11
Ped R/W Violation                                                8
Improper Passing                                                 8
Unsafe Lane Change                                               7
Following Too Closely                                            4
Other Hazardous Movement                                      

In [3]:
el_molino_lon_df.PartyType1.value_counts()

Driver            468
Bicyclist          14
Pedestrian          5
Parked Vehicle      2
Name: PartyType1, dtype: int64

In [6]:
el_molino_lon_df.groupby('Violation').Cause.value_counts()

ValueError: all keys need to be the same shape

In [7]:
driver_at_fault = el_molino_lon_df.loc[
    ((el_molino_lon_df['PartyType1'] == 'Driver') & (el_molino_lon_df['PtyAtFault'] == 1)) | 
    ((el_molino_lon_df['PartyType2'] == 'Driver') & (el_molino_lon_df['PtyAtFault'] == 2))
]
bicyclist_at_fault = el_molino_lon_df.loc[
    ((el_molino_lon_df['PartyType1'] == 'Bicyclist') & (el_molino_lon_df['PtyAtFault'] == 1)) | 
    ((el_molino_lon_df['PartyType2'] == 'Bicyclist') & (el_molino_lon_df['PtyAtFault'] == 2))
]
pedestrian_at_fault = el_molino_lon_df.loc[
    ((el_molino_lon_df['PartyType1'] == 'Pedestrian') & (el_molino_lon_df['PtyAtFault'] == 1)) | 
    ((el_molino_lon_df['PartyType2'] == 'Pedestrian') & (el_molino_lon_df['PtyAtFault'] == 2))
]

In [8]:
driver_at_fault.groupby('Violation').Cause.value_counts()

ValueError: all keys need to be the same shape

In [9]:
bicyclist_at_fault.groupby('Violation').Cause.value_counts()

ValueError: all keys need to be the same shape

In [10]:
pedestrian_at_fault.groupby('Violation').Cause.value_counts()

Violation  Cause               
21451      Pedestrian Violation    1
21950      Pedestrian Violation    1
21955      Pedestrian Violation    1
21956      Pedestrian Violation    1
Name: Cause, dtype: int64

In [11]:
el_molino_lon_df.InvWith.value_counts()

Other Motor Vehicle               351
Fixed Object                       43
Parked Motor Vehicle               36
Pedestrian                         23
Bicycle                            20
Not Stated                         10
Motor Vehicle on Other Roadway      2
Other Object                        2
Other                               1
Non-Collision                       1
Name: InvWith, dtype: int64

In [12]:
el_molino_lon_df.groupby('Cause').Violation.value_counts()

ValueError: all keys need to be the same shape

In [13]:
bike_involved = el_molino_lon_df.loc[el_molino_lon_df['InvWith'] == 'Bicycle']

In [14]:
len(pedestrian_at_fault)

4

In [15]:
driver_at_fault.Cause.value_counts()

Traffic Signals and Signs                                      122
Unsafe Speed                                                    59
Improper Turning                                                51
Auto R/W Violation                                              41
Driving Under Influence                                         17
Unsafe Starting or Backing                                      14
Pedestrian Violation                                             9
Unsafe Lane Change                                               7
Ped R/W Violation                                                7
Improper Passing                                                 7
Wrong Side of Road                                               6
Automobile Right of Way                                          4
Unknown                                                          3
Following Too Closely                                            3
Other Hazardous Movement                                      

In [16]:
bicyclist_at_fault.Cause.value_counts()

Wrong Side of Road           5
Traffic Signals and Signs    3
Improper Passing             1
Auto R/W Violation           1
Improper Turning             1
Name: Cause, dtype: int64

In [17]:
len(driver_at_fault)

358

In [18]:
# Causes
el_molino_lon_df.columns

Index(['OBJECTID', 'Accidno', 'Date', 'Time', 'Day', 'Street', 'CrossSt',
       'Distance', 'Direction', 'X', 'Y', 'Injury', 'NoInjured', 'NoKilled',
       'PtyAtFault', 'HitAndRun', 'CollisnTyp', 'InvWith', 'PedAction',
       'PvtProp', 'Violation', 'Weather', 'RoadCond', 'Lighting', 'Controls',
       'PartyType1', 'PartyAge1', 'PartySex1', 'Direction1', 'Movement1',
       'SpecInfo1', 'Sobriety1', 'VehType1', 'VehYear1', 'VehMake1',
       'VehModel1', 'SpeedLim1', 'Damage1', 'SafetyEq1', 'PartyType2',
       'PartyAge2', 'PartySex2', 'Direction2', 'Movement2', 'SpecInfo2',
       'Sobriety2', 'VehType2', 'VehYear2', 'VehMake2', 'VehModel2',
       'SpeedLim2', 'Damage2', 'SafetyEq2', 'Cause', 'RoadSurf', 'geometry',
       'lat', 'lon', 'datetime'],
      dtype='object')

## Street/Cross Street Analysis

In [4]:
el_molino_st_df = df.loc[(df['Street'] == 'EL MOLINO AVE') | (df['CrossSt'] == 'EL MOLINO AVE')]

# aggregate
print(f'Total collisions {len(el_molino_st_df)}')
display(el_molino_st_df.agg({'NoInjured' : ['sum'], 'NoKilled': ['sum'], 'datetime': ['min', 'max']}))

# plot
fig = px.density_mapbox(el_molino_st_df, lat='lat', lon='lon', radius=4,
                        center=dict(lat=34.15127, lon=-118.1369), zoom=13,
                        mapbox_style="stamen-terrain", height=1000, )
fig.show()

Total collisions 574


,NoInjured,NoKilled,datetime
sum,307.0,2.0,NaT
min,NaN,NaN,2008-01-11
max,NaN,NaN,2020-12-24
